# Detection model export to tensorrt

In [1]:
!yolo export model="best_model_yolo11s.pt" format="engine" imgsz="640" device=0 int8=True 

Ultralytics 8.3.57 🚀 Python-3.10.12 torch-2.4.0 CUDA:0 (Orin, 7620MiB)
WARNING ⚠️ INT8 export requires a missing 'data' arg for calibration. Using default 'data=coco8.yaml'.
YOLO11s summary (fused): 238 layers, 9,413,187 parameters, 0 gradients, 21.3 GFLOPs

PyTorch: starting from 'best_model_yolo11s.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 5, 8400) (18.3 MB)

ONNX: starting export with onnx 1.17.0 opset 19...
ONNX: slimming with onnxslim 0.1.34...
ONNX: export success ✅ 41.9s, saved as 'best_model_yolo11s.onnx' (36.1 MB)

TensorRT: starting export with TensorRT 10.3.0...
[06/04/2025-09:58:10] [TRT] [I] [MemUsageChange] Init CUDA: CPU +2, GPU +0, now: CPU 826, GPU 5435 (MiB)
[06/04/2025-09:58:21] [TRT] [I] [MemUsageChange] Init builder kernel library: CPU +927, GPU +1073, now: CPU 1796, GPU 6521 (MiB)
[06/04/2025-09:58:22] [TRT] [I] ----------------------------------------------------------------
[06/04/2025-09:58:22] [TRT] [I] Input filename:   best_model_yol

In [2]:
import tensorrt
print(tensorrt.__version__)
assert tensorrt.Builder(tensorrt.Logger())

10.3.0


# Recognition Model Export

In [3]:
import sys
sys.path.append("Recognition Model")  # Replace with your actual path

In [5]:
import torch
import torch.onnx
from num_plate_ocr import Model

In [8]:
from num_plate_ocr import Model

# Initialize your model
model = Model()
model.eval()

# Load state_dict and remove 'module.' prefix if present
state_dict = torch.load("Recognition Model/num_plate_ocr.pth", map_location='cpu')
new_state_dict = {}

for k, v in state_dict.items():
    new_k = k.replace("module.", "") if k.startswith("module.") else k
    new_state_dict[new_k] = v

# Load corrected state_dict
model.load_state_dict(new_state_dict)
print("✅ Model loaded successfully")


No Transformation module specified
✅ Model loaded successfully


/tmp/ipykernel_6873/2311238740.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load("Recognition Model/num_plate_ocr.pth", map_location='cpu')


In [ ]:
dummy_input = torch.randn(1, 1, 64, 600)  # Adjust based on your model?s input shape

torch.onnx.export(
    model,
    dummy_input,
    "easyocr_license_plate.onnx",
    input_names=["input"],
    output_names=["output"],
    dynamic_axes={"input": {0: "batch_size"}, "output": {0: "batch_size"}},
    opset_version=11
)

print("ONNX model exported successfully")
